# Entraînement GNN sur CORD-19

In [3]:
pip install torch_geometric


  Using cached torch_geometric-2.7.0-py3-none-any.whl.metadata (63 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
Using cached torch_geometric-2.7.0-py3-none-any.whl (1.3 MB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)

   ----------------- ---------------------- 4/9 [aiohappyeyeballs]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 [aiohttp]
   ------------------------------- -------- 7/9 [aiohttp]
   ----------------------------------- ---- 8/9 [torch_geometric]
   ----------------------------------- ---- 8/9 [torch_geometric]
   ----------------------------------- ---- 8/9 [torch_geometric]
   ----------------------------------- ---- 8/9 [torch_geometric]
   ----------------------------------- ---- 8/9 [

In [2]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 453.5 kB/s eta 0:00:08
   ----- ---------------------------------- 0.5/4.0 MB 453.5 kB/s eta 0:00:08
   ------- -------------------------------- 0.8/4.0 MB 508.5 kB/s eta 0:00:07
   ---------- ----------------------------- 1.0/4.0 MB 621.7 kB/s eta 0:00:05
   ---------- ----------------------------- 1.0/4.0 MB 621.7 kB/s eta 0:00:05
   ------------ --------------------------- 1.3/4.0 MB 610.3 kB/s eta 0:00:05
   ------------ --------------------------- 1.3/4.0 MB 610.3 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import json
BASE_PATH = Path("../data/processed")
MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)
GRAPH_PATH = BASE_PATH / "article_graph.gpickle"
EMB_PATH = BASE_PATH / "embeddings.npy"
CSV_PATH = BASE_PATH / "articles_with_communities.csv"
HIDDEN_DIM = 256
EPOCHS = 50
LR = 0.001
DROPOUT = 0.3
PATIENCE = 8


In [3]:
import pickle

print("📂 Chargement des données...")

# --- Charger le graphe .gpickle ---
with open(GRAPH_PATH, "rb") as f:
    G = pickle.load(f)

# --- Embeddings et CSV ---
emb = np.load(EMB_PATH)
df = pd.read_csv(CSV_PATH)

labels = df['community'].astype(int).values

# --- Assurer cohérence des dimensions ---
n = min(len(df), emb.shape[0], G.number_of_nodes())

df = df.iloc[:n].reset_index(drop=True)
labels = labels[:n]
emb = emb[:n]

print(f"✅ {n:,} noeuds")
df.head(2)


📂 Chargement des données...


C:\Users\lapte\AppData\Local\Temp\ipykernel_22576\2583446052.py:11: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


✅ 816,359 noeuds


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,...,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id,year,full_text,text_length,community
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,...,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN,2001.0,Clinical features of culture-proven Mycoplasma...,1976,0
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,...,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN,2000.0,Nitric oxide: a pro-inflammatory mediator in l...,1060,1


In [4]:
print("🧱 Construction des tenseurs PyG...")
H = G.copy()
for i in range(n):
    if i not in H:
        H.add_node(i)
x = torch.tensor(emb, dtype=torch.float32)
y = torch.tensor(labels, dtype=torch.long)
d = from_networkx(H)
edge_index = d.edge_index
data = Data(x=x, edge_index=edge_index, y=y)
data.num_node_features, int(torch.unique(y).numel())


🧱 Construction des tenseurs PyG...


(384, 187)

In [1]:
print("🎯 Split train/val/test...")
idx = np.arange(n)
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
train_idx, rem_idx = next(sss1.split(idx, labels))
lab_rem = labels[rem_idx]
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
val_idx, test_idx = next(sss2.split(rem_idx, lab_rem))
train_mask = torch.zeros(n, dtype=torch.bool); train_mask[train_idx] = True
val_mask = torch.zeros(n, dtype=torch.bool); val_mask[val_idx] = True
test_mask = torch.zeros(n, dtype=torch.bool); test_mask[test_idx] = True
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
int(train_mask.sum()), int(val_mask.sum()), int(test_mask.sum())


🎯 Split train/val/test...


NameError: name 'np' is not defined

In [ ]:
print("🧠 Modèle...")
try:
    from step4_gnn import EfficientGNNClassifier
    model = EfficientGNNClassifier(input_dim=data.num_node_features, hidden_dim=HIDDEN_DIM, output_dim=int(torch.unique(y).numel()))
except Exception:
    from torch_geometric.nn import SAGEConv
    class EfficientGNNClassifier(nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim):
            super().__init__()
            self.conv1 = SAGEConv(input_dim, hidden_dim)
            self.conv2 = SAGEConv(hidden_dim, hidden_dim)
            self.lin = nn.Linear(hidden_dim, output_dim)
            self.drop = nn.Dropout(p=DROPOUT)
        def forward(self, data):
            x, edge_index = data.x, data.edge_index
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.drop(x)
            x = self.conv2(x, edge_index)
            x = torch.relu(x)
            x = self.drop(x)
            out = self.lin(x)
            return out
    model = EfficientGNNClassifier(input_dim=data.num_node_features, hidden_dim=HIDDEN_DIM, output_dim=int(torch.unique(y).numel()))
opt = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss()
model


In [ ]:
best_val = 0.0
best_state = None
hist = {"epoch": [], "train_loss": [], "val_acc": []}
no_improve = 0
for epoch in range(EPOCHS):
    model.train()
    opt.zero_grad()
    out = model(data)
    loss = loss_fn(out[data.train_mask], y[data.train_mask])
    loss.backward()
    opt.step()
    model.eval()
    with torch.no_grad():
        pred_val = out[data.val_mask].argmax(dim=1)
        acc_val = float((pred_val == y[data.val_mask]).sum().item()) / float(data.val_mask.sum().item())
    hist["epoch"].append(epoch)
    hist["train_loss"].append(float(loss.item()))
    hist["val_acc"].append(acc_val)
    if acc_val > best_val:
        best_val = acc_val
        best_state = {k: v.clone() for k, v in model.state_dict().items()}
        no_improve = 0
    else:
        no_improve += 1
    if no_improve >= PATIENCE:
        break
best_val


In [ ]:
if best_state is not None:
    model.load_state_dict(best_state)
model.eval()
with torch.no_grad():
    out = model(data)
pred_test = out[data.test_mask].argmax(dim=1).cpu().numpy()
true_test = y[data.test_mask].cpu().numpy()
acc_test = float((pred_test == true_test).sum()) / float(len(true_test))
report = classification_report(true_test, pred_test, output_dict=True)
acc_test


In [ ]:
plt.figure(figsize=(8,4))
plt.plot(hist['epoch'], hist['train_loss'], label='train_loss')
plt.plot(hist['epoch'], hist['val_acc'], label='val_acc')
plt.legend()
plt.title('Historique GNN')
plt.tight_layout()
plt.savefig('training_history_full.png', dpi=200, bbox_inches='tight')
plt.show()
cm = confusion_matrix(true_test, pred_test)
plt.figure(figsize=(6,5))
plt.imshow(cm, cmap='Blues')
plt.title('Matrice de confusion')
plt.colorbar()
plt.tight_layout()
plt.savefig('confusion_matrix_full.png', dpi=200, bbox_inches='tight')
plt.show()


In [ ]:
ckpt_path = MODEL_DIR / 'gnn_full_dataset.pt'
label_encoder = {"classes_": sorted(list(map(int, np.unique(labels))))}
torch.save({
    'model_state_dict': model.state_dict(),
    'test_acc': acc_test,
    'report': report,
    'label_encoder': label_encoder
}, ckpt_path)
str(ckpt_path)
